# Desafío 01

* Importación de librerías
* astype()

In [ ]:
#!pip install unidecode


In [1]:
import pandas as pd
import numpy as np
import re as re
import unidecode

In [2]:
# Para ver el contenido total de las columnas

pd.set_option('display.max_colwidth', -1)
#pd.set_option("display.max_columns", len(properatti.columns))

In [3]:
df = pd.DataFrame(pd.read_csv("../data/properatti_minusculas.csv", encoding='UTF-8'))
#df = pd.DataFrame(pd.read_csv("../data/properatti.csv", encoding='UTF-8'))


#!head nombreArchivo.cvs --> comando de linux para ver las 5 primeras lineas. Se puede utilizar para ver como estan separados los datos

In [ ]:
df.info(verbose=None, null_counts=True)

In [4]:
df.sample(1)['description']

57429    corredor responsable: pedro j. moldes - csi 6228contacto: sergio bonavia - mls id # 420851009-16departamento en construccion con fecha de finalizacion en septiembre de 2017. contrafrente de 2 ambientes con balcon y amenities: sum, parrilla, solarium. muy  bien ubicado en caballito, en una zona residencial con gran variedad de oferta gastronomica, comercial, colegios, facultades, rapida salida a las autopista. ubicado en el centro geografico de caba. el departamento tiene la cocina integrada con cocina a gas, espacio lavadero, agua caliente central, pisos de porcelanato, griferia fv, muebles de cocina (alacena y bajo mesada), mesada de granito, cortinas de enrollar en el dormitorio de aluminio, placard con puertas corredizas e interiores terminados, calefaccion por aire f/c instalado, cerramientos de aluminio. descubierta: 4 m2 superficie superficie cubierta: 45 m2 superficie total: 49 m2  orientacion: este luminosidad: muy luminoso apto profesional: si  transportes: colectivos

* Información de las columnas

In [ ]:
# Busqueda de NaNs
df[pd.isnull(df['description'])]


### Reemplazo de NaN's
Esto se realiza porque la función de búsqueda explotaba 

In [5]:
df_a1=df.replace(np.nan,'sin-dato')
#df_a1.head(2)

### Patterns

In [6]:
# Comentar lo que no se deba ejecutar

#pattern = r'(\d+)\s[m]2' # Patron para buscar metros cuadrados (m2)
pattern = r'(\d*\w*)\s*amb'  # Patron para buscar metros ambientes
#pattern = r'(\d*)\s?\b[aA]mbientes\w*'
#pattern = r'(\d*\w*)\s*usd'
#pattern = r'(\d*\w*)\s*habitacion|(\d*\w*)\s?cuarto|(\d*\w*)\s?dorm|(\d*\w*)\s?pieza'

In [7]:
regex = re.compile(pattern, flags = re.IGNORECASE)

### Pruebas de campos

In [50]:
df.iloc[55476]['description']

'departamento en primer piso por escalera, dos plantas. amplias vistas al lago gutierrez y los cerros. pb: hall frio. cocina integrada al living mediante barra con espacio para lavarropas. mueble secarropa. living comedor con salida a amplio balcon. 02 habitaciones, una en suite. toilette c/ducha. pa: suite principal con vestidor y bano con jacuzzi doble y ducha. pisos de madera plastificada y ventanas de pvc. incluye baulera y espacio guardacoche.'

### Funciones

In [8]:
# Busca claves segun pattern en columna dentro de dataFrame 
# Entrada: pattern, columna, df
# Salida: DataFrame

def busca_claves(pattern, columna, df_aux):
    regex = re.compile(pattern, flags = re.IGNORECASE)
    m = pd.DataFrame([regex.findall(n) for n in df_aux[columna].head(30)])
    return m[0]

In [9]:
def existe_clave(key, columna, df_aux):    
    m = []
    for frase in df_aux[columna]:
        if key in frase:
            m.append(True)
        else:
            m.append(False)
    return sum(m)


In [10]:
# No se está utilizando mas desde que el dataset se encuentra en minúsculas y sin acentos
def quitar_caracteres(column):
    for i in range(len(column)):
        if type(column[i]) == str:
            column[i] = str.lower(unidecode.unidecode(column[i]))

In [ ]:
quitar_caracteres(df["place_name"])
quitar_caracteres(df["place_with_parent_names"])
quitar_caracteres(df["country_name"])
quitar_caracteres(df["state_name"])
quitar_caracteres(df["description"])
quitar_caracteres(df["title"])


In [ ]:
existe_clave("$","description",df_a1)


### Concatenar DFs

In [ ]:
#pd.concat([df3, df4], axis='columns')

### TESTING Regex

In [11]:
df1 = busca_claves(pattern,'description',df_a1)
#df1 = busca_claves(pattern,'description',df_a1)
#df2 = busca_claves(pattern,'title',df_a1)

In [ ]:
#prod.apply(lambda x: pd.api.types.infer_dtype(x.values))

### Rutina para reemplazar literales a numeros

In [12]:
# Coloco nombre en la columna para luego hacer merge

df1.rename(columns={0:'ambientes'}, inplace=True)

0     2   
1     None
2     2   
3     3   
4     None
5     None
6     None
7     mono
8     dos 
9     None
10    None
11    1   
12    2   
13    3   
14        
15    None
16    4   
17    3   
18    los 
19    t   
20    None
21    None
22    los 
23    t   
24    4   
25    4   
26    None
27    None
28    4   
29    sus 
dtype: object

In [13]:
dic = {'mono': 1, 'un':1, 'uno':1, 'dos':2, 'tres':3, 'cuatro':4, 'cinco':5 ,'seis':6 ,'siete':7}
lista_prop = ['mono','un','uno','dos','tres','cuatro','cinco','seis','siete']

def cambiar_x_nros(df):
    aux1 = pd.DataFrame([x if x not in lista_prop else dic.get(x) for x in df])
    aux2 = pd.DataFrame([int(x) if int(x) == int else np.nan for x in aux1])
    return aux1

# Le voy a dar otra vuelta para reemplazar los no numericos por NaNs

In [47]:
# backup
dic = {'mono': 1, 'un':1, 'uno':1, 'dos':2, 'tres':3, 'cuatro':4, 'cinco':5 ,'seis':6 ,'siete':7}
lista_prop = ['mono','un','uno','dos','tres','cuatro','cinco','seis','siete']

def cambiar_x_nros(df):
    lista = [x if x not in lista_prop else dic.get(x) for x in df]
    lista2 = pd.to_numeric(lista, errors='ignore', downcast='float')
    #lista3 = [x[0] if x[0] == float else np.nan for x in lista2]
    return lista2
    
# Le voy a dar otra vuelta para reemplazar los no numericos por NaNs

In [63]:
dfa1 = cambiar_x_nros(df1)
[pd.to_numeric(x, errors='ignore', downcast='float') for x in dfa1]


#dfa1[0][1]
#type(dfa1[0][0])



[2.0,
 nan,
 2.0,
 3.0,
 nan,
 nan,
 nan,
 1,
 2,
 nan,
 nan,
 1.0,
 2.0,
 3.0,
 nan,
 nan,
 4.0,
 3.0,
 'los',
 't',
 nan,
 nan,
 'los',
 't',
 4.0,
 4.0,
 nan,
 nan,
 4.0,
 'sus']

In [53]:
dfa1

array(['2', None, '2', '3', None, None, None, 1, 2, None, None, '1', '2',
       '3', '', None, '4', '3', 'los', 't', None, None, 'los', 't', '4',
       '4', None, None, '4', 'sus'], dtype=object)

In [42]:
pd.to_numeric(dfa1[0][0])+2

4

In [28]:
dfa1


array(['2', None, '2', '3', None, None, None, 1, 2, None, None, '1', '2',
       '3', '', None, '4', '3', 'los', 't', None, None, 'los', 't', '4',
       '4', None, None, '4', 'sus'], dtype=object)

### Fin rutina 

In [ ]:
#df1['ambientes'] = df1['ambientes'].apply(lambda x: dic.get(x))
#df1

#df1['ambientes'].apply(lambda x: dic.get(x))
#df["Date"] = df["Member"].apply(lambda x: d.get(x))

In [ ]:
# DF para join muchos a muchos

df_conve = pd.DataFrame({'ambientes': ['mono','un','uno','dos','tres','cuatro','cinco','seis','siete'],
           'numero': [1,1,1,2,3,4,5,6,7]})
df_conve


In [ ]:
# Pasaje de cifras literales a nros

df2 = pd.merge(df1,df_conve)
df2

# SIN USO

In [ ]:
#tmp_df = df["cases"].str.extract("([a-z])([0-9]+)([0-9]{2})",expand=True) 
tmp_df = df["description"].str.extract("((\d*)\s?\b[aA]mb\w*)",expand=True) 
tmp_df.dropna(axis=0)

In [ ]:
# TEST: Regex de prueba. Alimenta un DF con el resultado
# CONDICION: No lee NaN's
# NOTA: Me di cuenta que REGEX analiza todo el str y en muchos casos detecta la KEY en varias partes del 
#   str, ergo, al capturar el siguiente dato, lo incluye dentro de una lista y eso lo representa en otra columna
#   Entonces ponerle nombre a una única columna no tendría sentido. Si verificamos con .head(10) no vamos
#   a tener problemas porque justo las 10 primeras filas no tienen mas de una KEY en el str a analizar...

#m = pd.DataFrame([regex.findall(n) for n in df_a1['description'].head(10)], columns=["ambientes"] )
m = pd.DataFrame([regex.findall(n) for n in df_a1['description'].head(50)])
#m = pd.DataFrame([regex.findall(n) for n in df_a1['description']])
display(len(m))
#m
m[0]


In [ ]:
# Para pasar una linea a un tipo en particular:

c = "1"
pd.to_numeric(c,int)